In [2]:
import scib
import scanpy as sc
import numpy as np

In [3]:
adata_after = sc.read_h5ad('./after.h5ad')
adata_after

/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/h5py/_hl/dataset.py:767: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)


AnnData object with n_obs × n_vars = 14035 × 1399
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'UMAP_1', 'UMAP_2', 'Celltype..malignancy.', 'Celltype..major.lineage.', 'Celltype..minor.lineage.', 'Cluster', 'Patient', 'batch'
    var: 'features', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'X_name', 'hvg', 'ruvK', 'ruvK_optimal', 'scRep_res', 'timeReplicates', 'timeRuv'
    layers: 'X', 'counts', 'logcounts'

In [4]:
adata_after.X.shape

(14035, 1399)

In [5]:
adata = adata_after.copy()
adata.X = np.array(adata_after.layers['X'].todense())
adata

AnnData object with n_obs × n_vars = 14035 × 1399
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'UMAP_1', 'UMAP_2', 'Celltype..malignancy.', 'Celltype..major.lineage.', 'Celltype..minor.lineage.', 'Cluster', 'Patient', 'batch'
    var: 'features', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'X_name', 'hvg', 'ruvK', 'ruvK_optimal', 'scRep_res', 'timeReplicates', 'timeRuv'
    layers: 'X', 'counts', 'logcounts'

In [28]:
sc.pp.neighbors(adata,use_rep='X')
sc.tl.diffmap(adata)

adata.uns['iroot'] = 9999
# adata.var['xroot']= np.array(adata.X.todense()).mean(axis=0)
sc.tl.dpt(adata)
adata.obs['dpt_pseudotime']

AAACCTGGTTATGCGT-1-0            0.077236
AAACGGGCAGCTTAAC-1-0            0.197089
AAACGGGCAGTACACT-1-0            0.033106
AAACGGGCATACAGCT-1-0            0.182526
AAAGATGAGGGTTCCC-1-0            0.039814
                                  ...   
CART_RP@GACTACACAACGCACC-1      0.010483
CART_RP@CCTAAAGAGAAGCCCA-1      0.022767
Tcells_RP@CTAAGACTCTTCCTTC-1    0.022385
Tcells_RP@GGGACCTTCTTGTACT-1    0.016870
CART_RP@CACATTTAGCTCTCGG-1      0.019521
Name: dpt_pseudotime, Length: 14035, dtype: float32

In [29]:
sc.pp.neighbors(adata_after,use_rep='X')
sc.tl.diffmap(adata_after)

adata_after.uns['iroot'] = 0
sc.tl.dpt(adata_after)
adata_after.obs['dpt_pseudotime']

AAACCTGGTTATGCGT-1-0            0.000000
AAACGGGCAGCTTAAC-1-0            0.000041
AAACGGGCAGTACACT-1-0            0.000416
AAACGGGCATACAGCT-1-0            0.000206
AAAGATGAGGGTTCCC-1-0            0.000048
                                  ...   
CART_RP@GACTACACAACGCACC-1      0.000318
CART_RP@CCTAAAGAGAAGCCCA-1      0.000057
Tcells_RP@CTAAGACTCTTCCTTC-1    0.000825
Tcells_RP@GGGACCTTCTTGTACT-1    0.000457
CART_RP@CACATTTAGCTCTCGG-1      0.000159
Name: dpt_pseudotime, Length: 14035, dtype: float32

In [30]:
scib.me.trajectory_conservation(adata, adata_after, label_key="Celltype..major.lineage.")


/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/scib/metrics/trajectory.py:120: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  n_components, adata_post.obs["neighborhood"] = connected_components(
/Users/dongjiajie/opt/anaconda3/lib/python3.8/contextlib.py:120: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)


0.5986280640660694

In [18]:
get_root(adata_after,adata,"Celltype..major.lineage.")

/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/scanpy/tools/_dpt.py:18: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata.obsm['X_diffmap'] = dpt.eigen_basis


(5306,
 AnnData object with n_obs × n_vars = 14035 × 1399
     obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'UMAP_1', 'UMAP_2', 'Celltype..malignancy.', 'Celltype..major.lineage.', 'Celltype..minor.lineage.', 'Cluster', 'Patient', 'batch', 'dpt_pseudotime', 'neighborhood'
     var: 'features', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
     uns: 'X_name', 'hvg', 'ruvK', 'ruvK_optimal', 'scRep_res', 'timeReplicates', 'timeRuv', 'neighbors', 'diffmap_evals', 'iroot'
     obsm: 'X_diffmap'
     layers: 'X', 'counts', 'logcounts'
     obsp: 'distances', 'connectivities')

In [16]:
from scipy.sparse.csgraph import connected_components

def get_root(adata_pre, adata_post, ct_key, pseudotime_key="dpt_pseudotime", dpt_dim=3):
    
    """Determine root cell for integrated adata based on unintegrated adata

    :param adata_pre: unintegrated adata
    :param adata_post: integrated adata
    :param label_key: column in ``adata_pre.obs`` of the groups used to precompute the trajectory
    :param pseudotime_key: column in ``adata_pre.obs`` in which the pseudotime is saved in.
        Column can contain empty entries, the dataset will be subset to the cells with scores.
    :param dpt_dim: number of diffmap dimensions used to determine root
    """
    n_components, adata_post.obs["neighborhood"] = connected_components(
        csgraph=adata_post.obsp["connectivities"], directed=False, return_labels=True
    )

    start_clust = adata_pre.obs.groupby(ct_key)[pseudotime_key].mean().idxmin()
    min_dpt = adata_pre.obs[adata_pre.obs[ct_key] == start_clust].index
    which_max_neigh = (
        adata_post.obs["neighborhood"]
        == adata_post.obs["neighborhood"].value_counts().idxmax()
    )
    min_dpt = [
        value for value in min_dpt if value in adata_post.obs[which_max_neigh].index
    ]

    adata_post_ti = adata_post[which_max_neigh]

    min_dpt = [adata_post_ti.obs_names.get_loc(i) for i in min_dpt]

    # compute Diffmap for adata_post
    sc.tl.diffmap(adata_post_ti)

    # determine most extreme cell in adata_post Diffmap
    min_dpt_cell = np.zeros(len(min_dpt))
    for dim in np.arange(dpt_dim):

        diffmap_mean = adata_post_ti.obsm["X_diffmap"][:, dim].mean()
        diffmap_min_dpt = adata_post_ti.obsm["X_diffmap"][min_dpt, dim]

        # count opt cell
        if len(diffmap_min_dpt) == 0:
            raise RootCellError("No root cell in largest component")

        # choose optimum function
        if len(diffmap_min_dpt) > 0 and diffmap_min_dpt.mean() < diffmap_mean:
            opt = np.argmin
        else:
            opt = np.argmax

        min_dpt_cell[opt(diffmap_min_dpt)] += 1

    # root cell is cell with max vote
    return min_dpt[np.argmax(min_dpt_cell)], adata_post_ti